In [1]:
import pandas as pd
import sqlite3 as sql

db = '../database_data/shows.db'
conn = sql.connect(db)
cur = conn.cursor()

In [2]:
#import all csv files

actors = pd.read_csv("../database_data/populate_data/actors.csv")
users = pd.read_csv("../database_data/populate_data/users.csv")
reviews = pd.read_csv("../database_data/populate_data/reviews.csv")
genre = pd.read_csv("../database_data/populate_data/genre.csv")
rls_table = pd.read_csv("../database_data/populate_data/rls_table.csv")
movie = pd.read_csv("../database_data/populate_data/movie.csv")
genre_shows = pd.read_csv("../database_data/populate_data/movie_show.csv")
tv_shows = pd.read_csv("../database_data/populate_data/shows.csv")
all_comments = pd.read_csv("../database_data/populate_data/all_comment_info.csv")
comment_reply = pd.read_csv("../database_data/populate_data/comment_reply.csv")
down = pd.read_csv("../database_data/populate_data/down_vote.csv")
up = pd.read_csv("../database_data/populate_data/up_vote.csv")

In [3]:
# small adjustments made to csv files to allign with the database schema
del tv_shows["Unnamed: 0"]
del genre_shows["Unnamed: 0"]
del genre["Unnamed: 0"]
del movie["Unnamed: 0"]
genre_shows.rename(columns = {'movie_show_id':'show_id' }, inplace = True)
actors.rename(columns = {'age':'gender', "ID": "actor_id"}, inplace = True)
reviews.rename(columns = {'comments_id':'review_id'}, inplace = True)
down.rename(columns = {'down_vote':'num_down_vote'}, inplace = True)
up.rename(columns = {'up_vote':'num_up_vote'}, inplace = True)
rls_table.rename(columns = {'shows_id':'show_id', "actors_id" : "actor_id"}, inplace = True)
reviews["ratings"] = reviews["ratings"].round(1)

## DATABASE CREATION

In [4]:
actors_query = """
                    CREATE TABLE actors (
                    actor_id INT NOT NULL PRIMARY KEY,
                    first_name TEXT NOT NULL,
                    last_name TEXT NOT NULL,
                    gender TEXT,
                    origin TEXT,
                    birth_dates TEXT
                    )"""

cur.execute(actors_query)
actors.to_sql('actors', con = conn, index = False, if_exists="append")

In [5]:
users_query = """
                    CREATE TABLE users (
                    user_id INTEGER NOT NULL PRIMARY KEY,
                    email_address TEXT NOT NULL,
                    name TEXT,
                    password TEXT
                    )"""
cur.execute(users_query)
users.to_sql('users', con = conn, index = False, if_exists="append")


In [6]:
series_query = """
                    CREATE TABLE series (
                    show_id TEXT NOT NULL PRIMARY KEY,
                    title TEXT NOT NULL,
                    country TEXT,
                    release_year INTEGER,
                    is_adult TEXT,
                    num_seasons TEXT,
                    description TEXT, 
                    num_episodes INTEGER 
                    )"""

cur.execute(series_query)
tv_shows.to_sql('series', con = conn, index = False, if_exists="append")

In [7]:
movie_query = """
                    CREATE TABLE movie (
                    show_id TEXT NOT NULL PRIMARY KEY,
                    title TEXT NOT NULL,
                    country TEXT,
                    release_year INT,
                    is_adult TEXT,
                    duration INTEGER, 
                    description TEXT 
                    )"""

#make sure remove min in duration. can use triggers to only include in min
cur.execute(movie_query)
movie.to_sql('movie', con = conn, index = False, if_exists="append")

In [8]:
comment_info = """
                    CREATE TABLE comment_info (
                    comment_id TEXT NOT NULL PRIMARY KEY,
                    comment TEXT) """

cur.execute(comment_info)
all_comments.to_sql('comment_info', con = conn, index = False, if_exists="append")

In [9]:
reviews_query = """
                    CREATE TABLE review (
                    review_id INTEGER NOT NULL PRIMARY KEY,
                    show_id TEXT NOT NULL,
                    user_id INTEGER NOT NULL,
                    reviews TEXT,
                    ratings FLOAT,
                    FOREIGN KEY(user_id) REFERENCES users(user_id) ON DELETE CASCADE,
                    FOREIGN KEY(show_id) REFERENCES movie(show_id) ON DELETE CASCADE,
                    FOREIGN KEY(show_id) REFERENCES series(show_id) ON DELETE CASCADE
                    )"""

cur.execute(reviews_query)
reviews.to_sql('review', con = conn, index = False, if_exists="append")

In [10]:
down_vote = """
            CREATE TABLE down_vote (
                    review_id INTEGER NOT NULL PRIMARY KEY,
                    num_down_vote INTEGER NOT NULL,
                    FOREIGN KEY(review_id) REFERENCES review(review_id) ON DELETE CASCADE)"""

cur.execute(down_vote)
down.to_sql('down_vote', con = conn, index = False, if_exists="append")

In [11]:
up_vote = """
            CREATE TABLE up_vote (
                    review_id INTEGER NOT NULL PRIMARY KEY,
                    num_up_vote INTEGER NOT NULL,
                    FOREIGN KEY(review_id) REFERENCES review(review_id) ON DELETE CASCADE)"""

cur.execute(up_vote)
up.to_sql('up_vote', con = conn, index = False, if_exists="append")

In [12]:
comment_rls =  """
                    CREATE TABLE comment_rls (
                    parent_comment_id TEXT NOT NULL,
                    child_comment_id TEXT NOT NULL,
                    review_id INTEGER NOT NULL,
                    user_id INTEGER NOT NULL, 
                    FOREIGN KEY(parent_comment_id) REFERENCES comment_info(comment_id) ON DELETE CASCADE,
                    FOREIGN KEY(child_comment_id) REFERENCES comment_info(comment_id) ON DELETE CASCADE,
                    FOREIGN KEY(review_id) REFERENCES review(review_id) ON DELETE CASCADE,
                    FOREIGN KEY(user_id) REFERENCES users(user_id) ON DELETE CASCADE
                    )"""

cur.execute(comment_rls)
comment_reply.to_sql('comment_rls', con = conn, index = False, if_exists="append")

In [13]:
rls_query = """
                    CREATE TABLE actor_rls (
                    show_id TEXT NOT NULL,
                    actor_id INTEGER NOT NULL,
                    FOREIGN KEY(actor_id) REFERENCES actors(actor_id) ON DELETE CASCADE,
                    FOREIGN KEY(show_id) REFERENCES movie(show_id) ON DELETE CASCADE,
                    FOREIGN KEY(show_id) REFERENCES series(show_id) ON DELETE CASCADE
                    )"""

cur.execute(rls_query)
rls_table.to_sql('actor_rls', con = conn, index = False, if_exists="append")

In [14]:
genre_query = """
                    CREATE TABLE genre (
                    genre_type TEXT NOT NULL,
                    genre_id TEXT NOT NULL PRIMARY KEY
                    )"""

cur.execute(genre_query)
genre.to_sql('genre', con = conn, index = False, if_exists="append")

In [15]:
rls_genre_query = """
                    CREATE TABLE genre_rls (
                    show_id TEXT NOT NULL,
                    genre_id TEXT NOT NULL,
                    FOREIGN KEY(genre_id) REFERENCES series(genre_id) ON DELETE CASCADE,
                    FOREIGN KEY(show_id) REFERENCES movie(show_id) ON DELETE CASCADE,
                    FOREIGN KEY(show_id) REFERENCES series(show_id) ON DELETE CASCADE
                    )"""

cur.execute(rls_genre_query)
genre_shows.to_sql('genre_shows', con = conn, index = False, if_exists="append")

# VIEW Table Creation

View is created to accelerate common data retrival from a database. Hence, we'll be creating some to compliment query database applications.

In [16]:
movie_ratings_view = """
              CREATE VIEW IF NOT EXISTS movie_ratings AS
                SELECT m.show_id, m.title, ROUND(AVG(r.ratings),2) AS avg_rating, SUM(uv.num_up_vote - dv.num_down_vote) AS review_score
                FROM movie AS m
                LEFT JOIN review AS r ON r.show_id = m.show_id
                LEFT JOIN up_vote AS uv ON uv.review_id == r.review_id
                LEFT JOIN down_vote AS dv ON dv.review_id == r.review_id
                GROUP BY m.show_id;""" 

cur.execute(movie_ratings_view)

In [17]:
shows_ratings_view = """
              CREATE VIEW IF NOT EXISTS show_ratings AS
                SELECT s.show_id, s.title, ROUND(AVG(r.ratings),2) AS avg_rating
                FROM series AS s
                INNER JOIN review AS r ON s.show_id = r.show_id
                GROUP BY s.show_id;""" 

cur.execute(shows_ratings_view)

In [18]:
review_ratings = """
             CREATE VIEW IF NOT EXISTS review_ratings AS
                SELECT *, COUNT(rls.review_id) as num_comments, uv.num_up_vote, dv.num_down_vote, uv.num_up_vote - dv.num_down_vote AS review_vote
                FROM review AS r
                LEFT JOIN comment_rls AS rls ON rls.review_id == r.review_id 
                LEFT JOIN up_vote AS uv ON uv.review_id == r.review_id
                LEFT JOIN down_vote AS dv ON dv.review_id == r.review_id
                GROUP BY 1
                """

cur.execute(review_ratings)

# Triggers

#### Email Verification

In [19]:
email_validation = """
                CREATE TRIGGER validate_email_before_insert_users 
                BEFORE INSERT ON users
                BEGIN
                SELECT CASE
                    WHEN new.email_address NOT LIKE '%_@__%.__%' THEN
                    RAISE (ABORT,'Invalid email address')
                  END;
               END;"""

cur.execute(email_validation) #New account email checked

In [20]:
email_update = """
                CREATE TRIGGER validate_email_update 
                BEFORE UPDATE ON users
                BEGIN
                SELECT CASE
                    WHEN new.email_address NOT LIKE '%_@__%.__%' THEN
                    RAISE (ABORT,'Invalid email address')
                  END;
               END;"""

cur.execute(email_update) #Existing account updated email checked

#### New email logs

In [21]:
eml_log = """
          CREATE TABLE email_update_logs (
          id INTEGER PRIMARY KEY,
          user_id INTEGER,
          old_email TEXT,
          new_email TEXT,
          user_action TEXT,
          created_at TEXT,
          FOREIGN KEY (user_id) REFERENCES users(user_id));"""

cur.execute(eml_log) #added logs table to keep tracks of changes of account details 

In [22]:
email_before = """
        CREATE TRIGGER log_email_after_update AFTER UPDATE ON users
        WHEN old.email_address <> new.email_address
            BEGIN
            INSERT INTO email_update_logs (
                user_id,
                old_email,
                new_email,
                user_action,
                created_at)
            VALUES (
                user_id,
                old.email_address,
                new.email_address,
                'UPDATE',
                DATETIME('NOW')) ;
            END;"""

cur.execute(email_before) # logs old and new email

#### Account Creation

In [23]:
before_insert_account = """
                CREATE TRIGGER bef_insert_user BEFORE INSERT ON users
                BEGIN
                   SELECT CASE
                   WHEN ((SELECT u.email_address 
                          FROM users AS u 
                          WHERE u.email_address = NEW.email_address) IS NOT NULL)
                   THEN RAISE(FAIL, 'ERROR: This email address has been registered')
                   END;
                END;"""

cur.execute(before_insert_account) #new account cannot have used email address c

In [24]:
after_insert_account = """
        CREATE TRIGGER log_email_after_insert AFTER INSERT ON users
            BEGIN
            INSERT INTO email_update_logs (
                user_id,
                old_email,
                new_email,
                user_action,
                created_at)
            VALUES (
                new.user_id,
                "Not Available",
                 new.email_address,
                'New Account',
                DATETIME('NOW')) ;
            END;"""
cur.execute(after_insert_account) # logs new account 

#### New shows

In [25]:
before_insert_movie = """
                CREATE TRIGGER bef_insert_movie BEFORE INSERT ON movie
                BEGIN
                   SELECT CASE
                   WHEN NEW.duration < 40
                       THEN RAISE(FAIL, 'ERROR: Duration should be in minutes')
                   WHEN new.release_year > CAST(strftime('%Y', DATETIME("NOW")) AS INT)
                       THEN RAISE(FAIL, "ERROR: Movie cannot be released in the future")
                   END;
                END;"""

cur.execute(before_insert_movie)

In [26]:
before_insert_series = """
                CREATE TRIGGER bef_insert_series BEFORE INSERT ON series
                BEGIN
                   SELECT CASE
                   WHEN NEW.num_of_seasons > 40
                       THEN RAISE(FAIL, 'ERROR: Duration should be in seasons')
                   WHEN new.release_year > CAST(strftime('%Y', DATETIME("NOW")) AS INT)
                       THEN RAISE(FAIL, "ERROR: Series cannot be released in the future")
                   END;
                END;"""

cur.execute(before_insert_series)

#### Rating range

In [27]:
before_insert_review = """
                CREATE TRIGGER bef_insert_rating BEFORE INSERT ON review
                BEGIN
                   SELECT CASE
                   WHEN NEW.ratings >= 0 AND new.ratings <= 10 
                       THEN RAISE(FAIL, 'ERROR: ratings must be in between 0 and 10')
                   END;
                END;"""

cur.execute(before_insert_review) 

In [28]:
before_update_review = """
                CREATE TRIGGER bef_update_rating BEFORE UPDATE ON review
                BEGIN
                   SELECT CASE
                   WHEN NEW.ratings >= 0 AND new.ratings <= 10 
                       THEN RAISE(FAIL, 'ERROR: ratings must be in between 0 and 10')
                   END;
                END;"""

cur.execute(before_update_review)

#### Comment depth control

In [29]:
before_insert_comment =  """
                CREATE TRIGGER before_insert_comment 
                BEFORE INSERT ON comment_rls
                BEGIN
                SELECT CASE
                    WHEN new.parent_comment_id LIKE '%C-Reply-___$' THEN RAISE (ABORT,'Only one level sub comment is allowed') 
                  END;
               END;"""

cur.execute(before_insert_comment) #Avoid more than one level sub comment to happen. A parent comment should have an id that looks like C-___

# INDEX 

In [30]:
#create an index on movie title for faster data retrieval 
index_movie = """
                CREATE INDEX movie_index ON movie (title) """

#create an index on show title for faster data retrieval 
index_show = """
                CREATE INDEX series_index  ON series (title)""" 

cur.execute(index_movie)
cur.execute(index_show)

# Queries

In [31]:
# Main function for running SQL queries
def run_query(q):
    # Connect to the database
    with sql.connect(db) as conn:
        return pd.read_sql(q,conn)
    
# Function to show all the tables present in the database.
def show_tables():
    q = '''
        SELECT
            name
        FROM sqlite_master
        WHERE type IN ("table","view");
        '''
    return run_query(q)

# Function to count how many rows in each table
def get_table_row_count(tablename):
    q = '''
        SELECT
            COUNT(1)
        FROM %s;
        ''' % tablename
    return run_query(q)["COUNT(1)"][0]

tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables["name"]]

tables

,name,row_count
0,actors,800
1,users,9000
2,series,2676
3,movie,6131
4,comment_info,400006
5,review,50000
6,down_vote,50000
7,up_vote,50000
8,comment_rls,200057
9,actor_rls,176140


#### Top 100 Movies and Shows

In [32]:
top_show = """
            WITH shows_movies AS (
                    SELECT mr.show_id, mr.title, mr.avg_rating, "movie" AS type
                    FROM movie_ratings AS mr

                UNION
                    SELECT *, "show" AS type
                    FROM show_ratings AS sr

                )
                
                SELECT sm.title, sm.avg_rating, sm.type
                FROM shows_movies AS sm
                ORDER by sm.avg_rating DESC
                LIMIT 100"""

run_query(top_show)

,title,avg_rating,type
0,Scream 2,10.00,movie
1,Coach Snoop,9.90,show
2,Lost Girls,9.80,movie
3,We Belong Together,9.80,movie
4,The Wishing Tree,9.75,movie
...,...,...,...
95,Furie,8.33,movie
96,The Misadventures of Hedi and Cokeman,8.30,movie
97,The Dark Crystal: Age of Resistance,8.30,show
98,Yeh Hai Bakrapur,8.30,movie


#### Worst rated UK movies

In [33]:
worst_rate = """SELECT m.title, ROUND(AVG(r.ratings),2) AS avg_ratings
                FROM review AS r
                INNER JOIN movie AS m ON m.show_id = r.show_id
                WHERE m.country == "United Kingdom"
                GROUP BY r.show_id
                ORDER BY 2 ASC
                LIMIT 10"""

run_query(worst_rate)


,title,avg_ratings
0,Last Breath,0.60
1,Hot Property,0.60
2,Slow West,0.75
3,Figaro Pho,1.50
4,David Brent: Life on the Road,1.70
5,Secrets of Her Majesty's Secret Service,1.70
6,Effie Gray,1.83
7,The King,1.86
8,Shawn Mendes: Live in Concert,2.03
9,Sitting in Limbo,2.05


#### Top Reviews

In [34]:
top_review = """
                WITH all_show AS (
                    SELECT show_id, title, "Movie" AS type
                    FROM movie
                    UNION
                    SELECT show_id, title, "TV Show" AS type
                    FROM series
                )

                SELECT u.name, rr.reviews, als.title, als.type, rr.review_vote , rr.num_comments, (rr.review_vote + rr.num_comments)/2 AS engagement_score
                FROM review_ratings AS rr
                INNER JOIN users AS u ON u.user_id = rr.user_id
                INNER JOIN all_show AS als ON als.show_id = rr.show_id
                ORDER BY 7 DESC
                LIMIT 20;
                """

run_query(top_review)

,name,reviews,title,type,review_vote,num_comments,engagement_score
0,Ethan Hopkinson,I cannot believe the same guy directed this cr...,Samudri Lootere,Movie,8990,5,4497
1,Charlotte Stewart,This movie was hysterical. I haven't laughed t...,Return of the Prodigal Son,Movie,8993,2,4497
2,Catherine Blythe,I just saw this movie (mainly because Brady Co...,INDIA,Movie,8974,4,4489
3,Ron Lambert,"A message movie, but a rather good one. Outsta...",Latte and the Magic Waterstone,Movie,8970,6,4488
4,Luke Griffiths,Armageddon PPV<br /><br />The last PPV of 2006...,The Little Hours,Movie,8973,1,4487
5,Zara Chester,"And look how a true story, ""... with a little ...",Caida del Cielo,Movie,8972,1,4486
6,Julian Wood,Be careful with this one. Once you get yer mit...,Nature: Animals With Cameras,TV Show,8963,10,4486
7,Mason Fox,"You can call this one a flop, and that's a ver...",마녀사냥,TV Show,8965,2,4483
8,Molly Eddison,"Disjointed, unclear, bad screenplay, poor phot...",The 24 Hour War,Movie,8956,6,4481
9,Candice Ross,This is a very moving movie about life itself....,Disclosure,Movie,8959,1,4480


#### Movies with the most reviews

In [35]:
top_review_query = """
                    SELECT title, review_score FROM movie_ratings
                    ORDER BY review_score DESC
                    LIMIT 20
                    """

run_query(top_review_query)

,title,review_score
0,Honey 2,102500
1,David Batra: Elefanten i rummet,94770
2,Ice Guardians,82086
3,Jada,71905
4,"Blue, Painful, Fragile",70071
5,Tango With Me,69120
6,Cold Mountain,67987
7,The Arbitration,66352
8,Uncle Naji in UAE,65802
9,Bombay Talkies,65709


#### Movies with the most comment engagement 

In [36]:
engagement = """
    SELECT m.show_id , m.title, COUNT(rls.review_id) as engagement_score
    FROM movie AS m
    INNER JOIN review AS r ON r.show_id == m.show_id
    INNER JOIN comment_rls AS rls ON rls.review_id == r.review_id
    GROUP BY m.show_id
    ORDER BY 3 DESC
    LIMIT 20
"""
run_query(engagement)

,show_id,title,engagement_score
0,s1086,Night in Paradise,93
1,s2366,Alexandria ... Why?,88
2,s6310,Biggie & Tupac,86
3,s503,Tango With Me,84
4,s1672,The App That Stole Christmas,80
5,s2687,Chris D'Elia: No Pain,79
6,s7995,Shakti: The Power,77
7,s2324,The Mirror Boy,77
8,s465,Gunpowder Milkshake,76
9,s2720,David Batra: Elefanten i rummet,76


#### List of reviews of a selected show. 

In [37]:
# A function to retrieve list of review
def search_show_review(show_type, show_name):
        """
        input : show_type (str) is either series/movie
                show_name (str) is the movie/series name
        
        output :  list of reviews retrieved from the database """
        try:
                review_show = f"""
                        SELECT a.user_id, a.name, a.email_address, r.reviews,  r.ratings, r.num_up_vote, r.num_down_vote, r.num_comments
                        FROM review_ratings AS  r
                        INNER JOIN users AS a ON a.user_id = r.user_id 
                        WHERE r.show_id IN (SELECT m.show_id
                                        FROM {show_type} AS m
                                        WHERE m.title = "{show_name}" )
                        ORDER BY 2"""
        
        except sql.Error as error:
                print("Failed to update sqlite table", error)
        
        return  run_query(review_show)

In [38]:
search_show_review("movie", 'Night in Paradise')

,user_id,name,email_address,reviews,ratings,num_up_vote,num_down_vote,num_comments
0,5228,Brad Raven,Brad_Raven2979@bretoux.com,"Beginning where it left off, Doctor Who, Rose,...",7.4,8901,351,2
1,5412,Camila Bradshaw,Camila_Bradshaw6509@supunk.biz,"Elia Kazan, one of the best theater directors ...",6.9,4929,346,4
2,2214,Denis Horton,Denis_Horton1671@gmail.com,"Two houses, one street, one phone booth, one c...",0.0,763,398,2
3,3255,Faith Alldridge,Faith_Alldridge1379@sveldo.biz,At the surface COOLEY HIGH is a snappy ensembl...,5.5,8369,73,4
4,1231,Florence Bingham,Florence_Bingham2112@irrepsy.com,"Out of the many films I've seen, Tommy Boy is ...",9.0,2793,369,2
5,5602,Joseph Cunningham,Joseph_Cunningham8210@acrit.org,Just reading why this show got canceled makes ...,7.4,4123,394,4
6,881,Marla Shepherd,Marla_Shepherd8427@eirey.tech,This movie was a suprise for me while I was su...,5.3,347,3,3
7,89,Morgan Hogg,Morgan_Hogg3936@dionrab.com,"""Ashes of Time"" was an audacious project but e...",0.4,5256,260,13
8,2532,Nicholas Campbell,Nicholas_Campbell8498@womeona.net,This is an excellent film and one should not b...,5.0,1423,84,9
9,2952,Noah Tate,Noah_Tate3084@bungar.biz,This is one of the most satisfying of the book...,5.1,3405,324,12


In [39]:
search_show_review("series", "Pokémon: Indigo League")

,user_id,name,email_address,reviews,ratings,num_up_vote,num_down_vote,num_comments
0,3667,Bryce Parr,Bryce_Parr8078@eirey.tech,I watched this movie for the first time a few ...,1.8,8920,224,5
1,2641,Catherine Blackburn,Catherine_Blackburn2969@vetan.org,... or was Honest Iago actually smirking at th...,9.3,1556,101,1
2,2592,Georgia Vaughn,Georgia_Vaughn5298@brety.org,Wow. Simply awful. I was a fan of the original...,0.8,6407,323,36
3,8382,Mason Willson,Mason_Willson4381@typill.biz,This movie is one of my very favorites. It's h...,8.8,2163,243,6
4,336,Peter Coleman,Peter_Coleman3346@deons.tech,This movies is the best movie to watch for com...,5.2,2078,231,7


#### List of actors of a selected show

In [40]:
# A function to retrieve list of actors
def search_show_actor(show_type, show_name):
    """
    input : show_type (str) is either series/movie
            show_name (str) is the movie/series name
    
    output :  list of actors retrieved from the database """

    try:
        actor_list = f"""
                    SELECT a.*
                    FROM actor_rls AS rt
                    INNER JOIN actors AS a ON a.actor_id = rt.actor_id
                    WHERE rt.show_id IN (SELECT m.show_id
                                         FROM {show_type} AS m
                                         WHERE m.title = "{show_name}")"""
        
    except sql.Error as error:
                print("Failed to update sqlite table", error)
              
        
    return  run_query(actor_list)

In [41]:
search_show_actor("series", "Pokémon: Indigo League")

,actor_id,first_name,last_name,gender,origin,birth_dates
0,715,Sarah,Watson,Female,Seychelles,2/26/1952
1,534,Rick,Ebbs,Male,Bhutan,8/29/1956
2,276,Freya,Clark,Female,Chad,11/28/1955
3,709,Gloria,Morrow,Female,Namibia,6/20/1962
4,410,Kieth,Wren,Male,Belize,8/29/1973
5,463,Hayden,Yarlett,Male,Papua New Guinea,7/12/1952
6,627,Maria,Atkinson,Female,Kosovo,7/6/1973
7,249,Chelsea,Walsh,Female,Vatican City,10/19/1977
8,766,Jennifer,Goodman,Female,Poland,7/1/1985
9,387,Cassidy,Benfield,Female,Armenia,4/10/1986


In [42]:
search_show_actor("movie", "Samudri Lootere")

,actor_id,first_name,last_name,gender,origin,birth_dates
0,442,Maxwell,Vollans,Male,Ghana,12/29/1966
1,387,Cassidy,Benfield,Female,Armenia,4/10/1986
2,57,Russel,Mcguire,Male,China,7/15/1962
3,494,Evie,Murray,Female,Turkmenistan,1/5/1972
4,601,Shannon,Russell,Female,Senegal,11/29/1953
5,561,Stella,Oliver,Female,Vanuatu,12/18/1975
6,515,Judith,Dempsey,Female,Angola,2/27/1972
7,306,Roger,Cox,Male,Luxembourg,1/20/1970
8,245,Sofia,Rees,Female,Burkina Faso,7/18/1976
9,169,Carl,Reid,Male,Turkmenistan,9/22/1970


# Update

In [43]:
def updateSqliteTable(table, column, condition, identifier, column_change):
    try:

        sql_update_query = f"""UPDATE {table} SET {column} = ? WHERE {condition} = ?""" #sql
        data = (column_change, identifier)
        cur.execute(sql_update_query, data)
        conn.commit()
        print("Record Updated successfully")
        

    except sql.Error as error:
        print("Failed to update sqlite table", error)

Update function application. The function can be used to update any cell in the database just by specifying the table, attributes and conditions

In [44]:
run_query("SELECT * FROM up_vote LIMIT 10")

,review_id,num_up_vote
0,1,1016
1,2,1762
2,3,4484
3,4,1972
4,5,5884
5,6,3573
6,7,855
7,8,3108
8,9,4420
9,10,2828


In [45]:
updateSqliteTable("up_vote", "num_up_vote", "review_id", 1, 1010)

Record Updated successfully


In [46]:
run_query("SELECT * FROM up_vote LIMIT 10")

,review_id,num_up_vote
0,1,1010
1,2,1762
2,3,4484
3,4,1972
4,5,5884
5,6,3573
6,7,855
7,8,3108
8,9,4420
9,10,2828


Insert function that allows us to update a new row in any chosen table.

In [47]:
def insertSqliteTable(table, columns, values):
    try:

        sql_update_query = f"""INSERT INTO {table} {columns} VALUES {values}""" #sql
        cur.execute(sql_update_query)
        conn.commit()
        print("Record Updated successfully")
        

    except sql.Error as error:
        print("Failed to update sqlite table", error)

In [48]:
insertSqliteTable("users", ("user_id", "email_address", "name","password"), (99999, "abu2@gmail.com", "Abu",  "3452hfR"))

Record Updated successfully


In [49]:
run_query("""SELECT * FROM users WHERE user_id = 99999 """)

,user_id,email_address,name,password
0,99999,abu2@gmail.com,Abu,3452hfR


In [50]:
cur.close()
conn.close()